In [1]:
a = open("Articles.txt",'r')
b = a.read()
articles=[]
text=""
title=""
titles=[]
for i in b.split("-------------------"):
    if i =="":
        continue
    article = i.split('\n\n\n\n\n')
    text = article[1].strip('\n').strip(' ')
    if text.count('\n') <10:
        continue
    if text.count('https://') >10:
        continue   
    title = article[0].strip('\n').strip(' ')
    if title in titles:
        continue
    titles.append(title)
    articles.append([title,text])
a.close()

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
ps = SnowballStemmer('english')
wordRepArticles =[]
idf={}
articleslen=[]
for i in articles:
    title,text = i[0],i[1]
    text = text.replace('\\','');text = text.replace('.\n','\n');text = text.replace('\n\n','\n')
    articledist={}
    sentencelis=[]
    count=0
    for i in (text+'\n'+title).split('\n'):
        if i ==title:
            mult =2
        else:
            mult =1
        linedist={}
        v = i.split(' ')
        if len(v)<4:
            continue
        for word in v:
            if word not in stopwords.words('english'):
                word = word.replace('(','');word = word.replace(')','')
                word = word.replace('"','');word = word.replace(',','');word = word.strip(' ')
                stemmed = ps.stem(word)
                if stemmed in articledist.keys():
                    articledist[stemmed]+=mult
                else:
                    articledist[stemmed]=mult
                if stemmed in linedist.keys():
                    linedist[stemmed]+=1
                else:
                    linedist[stemmed]=1
            count+=1
        sentencelis.append([i,linedist])
    for words in articledist.keys():   
        if words in idf.keys():
            idf[words]+=1
        else:
            idf[words]=1
    wordRepArticles.append([title,articledist,sentencelis])
    articleslen.append(count)

In [3]:
print(len(wordRepArticles))
# wordRepArticles[0]
print(len(articles))

4335
4335


In [4]:
articles[:5]

[["Senegal gets Africa's first NBA training center",
  'Lagos (CNN) The National Basketball Association (NBA) on Monday commissioned a new training center in Saly, Senegal, the first of its kind in Africa, as part of its drive to attract young teenagers to the league.\n\nThe new facility, which features two indoor basketball courts, a multidimensional activity center, a weight room, conference rooms as well as dormitories and educational facilities, will be the primary training location for NBA Academy Africa future hopefuls from across the continent, the league said in a statement.\n\n"We are thrilled to open this new world-class venue in Saly for NBA Academy Africa prospects," said Amadou Gallo Fall, vice president and managing director of NBA Africa.\n\n"This facility will provide aspiring players from across the continent with state-of-the-art courts and training equipment, under the tutelage of our NBA coaches and our world-class academic staff," he said.\n\nThe NBA Academy Africa

In [5]:
import math
n = len(articles)
avgArticleLen = sum(articleslen)*1.0/n
match = [[0 for i in range(n)] for i in range(n)]
for i in range(n):
    mainarticle = wordRepArticles[i]
    mainlength = articleslen[i]
    for j in range(i+1,n):
        comparearticle = wordRepArticles[j]
        comparelength = articleslen[j]
        commonwords = list(set(mainarticle[1].keys()) & set(comparearticle[1].keys())) 
        value = 0
        for word in commonwords:
            mainvalue = (1+math.log(mainarticle[1][word]))*1.0/(math.log(1+mainlength/avgArticleLen))
            compvalue = (1+math.log(comparearticle[1][word]))*1.0/(math.log(1+comparelength/avgArticleLen))
            value += mainvalue*compvalue* math.log(1+n/idf[word])
        match[i][j] = value
        match[j][i] = value

In [6]:
topmatches = [[] for i in range(n)]
matches = 3
for i in range(n):
    vals = [[match[i][j],j] for j in range(n)]
    toparticle = sorted(vals,reverse=True)[:matches]
    print("Main title: "+ articles[i][0])
    for j in range(matches):
        topmatches[i].append(toparticle[j])
        print(str(j)+"/" + str(matches)+ ": " + articles[toparticle[j][1]][0])

Main title: Senegal gets Africa's first NBA training center
0/3: LA, Dallas, NY-NJ , DC Headline XFL Cities for First Season
1/3: NHL Greenlights Seattle Expansion Franchise
2/3: NBA Draft 2019 News
Main title: Police arrest 6 after a teenage boy died in an adulthood initiation ritual in Kenya
0/3: California teacher arrested for forcibly cutting boy's hair
1/3: Sister of attacked teenage Syrian refugee also targeted by bullies in UK school
2/3: Wilfred DeFour, 100-year-old Tuskegee Airman, dies
Main title: At least 35 people are dead and dozens are missing after a boat accident in Uganda
0/3: 6 dead, more hurt in nightclub stampede in Italy
1/3: Shock Video: Migrants Land Boat On California Beach 100 Miles From Border
2/3: Woman killed on train may have been leaning out of window, say police
Main title: US Embassy in DRC warns of "possible terrorist threat" against US facilities
0/3: 2nd deadliest Ebola outbreak in history spreads to major city
1/3: Ebola cases surge in Congo, with wo

Main title: Hundreds arrested in third weekend of Paris protests against rising fuel prices
0/3: France considers 'all options' to quell violent protests
1/3: Mass Arrests In Paris As ‘Yellow Vest’ Protests Enter Violent Third Week, Spread To Belgium
2/3: In Pictures: Yellow Vest 'Act IV' Protest Violence Spreads Across France
Main title: British boxer sends message of support to bullied Syrian refugee
0/3: Sister of attacked teenage Syrian refugee also targeted by bullies in UK school
1/3: UK student to be charged after Syrian refugee attacked
2/3: Tommy Robinson threatened with legal action over 'bully' video
Main title: Escaped snow leopard shot and killed in zoo
0/3: Asian elephant born at Ohio zoo was conceived via artificial insemination, officials say
1/3: Massive lion fends off more than 20 hyenas during attack, wild video shows
2/3: With 1 bullet, a police officer is shot in the face and a suspect is shot dead during a struggle: Officials
Main title: Left to go it alone, Ukrai

0/3: 6 of the safest cars on the road
1/3: Traffic accidents are eighth leading cause of death globally, according to WHO
2/3: Drake dominates Apple Music's top tracks of 2018
Main title: Iran’s Chain Murders: A wave of killings that shook a nation
0/3: Students help save bus driver after collapse
1/3: Man pleads guilty to ramming pedestrians in Australia
2/3: Triple child murders: The lodger who destroyed a family
Main title: Viewpoint: Jameela Jamil on why airbrushing should be illegal
0/3: Viewpoint: 'Why engagement rings objectify women'
1/3: "Snapchat dysmorphia": Selfies, photo filters driving people to plastic surgery, doctors say
2/3: 100 Women: Freedom Trash Can project
Main title: 'Rejected by my family, raped
0/3: As a teen, I would have welcomed gay conversion therapy. Thankfully, I dodged that trauma.
1/3: As Tanzania's LGBT fear for their lives, HIV will thrive
2/3: Fmr. Principal, 90, Arrested For Rape Of Students In The 1960s « CBS Baltimore
Main title: Wilder v Fury: T

Main title: Tyson Fury comes out of Wilder contest a winner despite contested draw | Kevin Mitchell
0/3: Wilder v Fury: Tyson Fury says 'world knows who real champion is' after draw
1/3: Deontay Wilder vs.Tyson Fury: A fight for salvation for two heavyweights
2/3: Weekend in sport: Arsenal, Liverpool triumph in thrilling EPL derbies; no fury in controversial boxing draw; Kiwis stroll to rugby win
Main title: Jordan Pickford apologises after mistake leads to Liverpool winner
0/3: Jurgen Klopp celebration: One for the ages or 'absolutely shocking' behaviour?
1/3: Manchester City’s Georgia Stanway inflicts first WSL defeat on Arsenal
2/3: Weekend in sport: Arsenal, Liverpool triumph in thrilling EPL derbies; no fury in controversial boxing draw; Kiwis stroll to rugby win
Main title: Manchester City’s Georgia Stanway inflicts first WSL defeat on Arsenal
0/3: Man City's unbeaten start to title defense ends at Chelsea
1/3: Watch Mohamed Salah torch Bournemouth defenders to score a hat trick


Main title: Pinkerton: George H.W. Bush and the Generation Touched with Fire
0/3: Former President George H.W. Bush's funeral in Texas: Live stream
1/3: How George H.W. Bush survived a harrowing brush with death in World War II
2/3: CNN to Live Stream the Funeral Service of Former President George H.W. Bush
Main title: Left Attacks George H.W. Bush After His Death: ‘Blatant Racism and Toxic Masculinity,’ ‘F**k Him’
0/3: Bill Bennett: George H.W. Bush Had 'A Trumpian Side'
1/3: CNN to Live Stream the Funeral Service of Former President George H.W. Bush
2/3: The Todd Starnes Show – December 4th, 2018
Main title: Netanyahu: We Will Always Remember Bush’s Commitment to Israel
0/3: U.S. Jewish Groups: Bush Was ‘Great Friend of Jewish People’
1/3: CNN to Live Stream the Funeral Service of Former President George H.W. Bush
2/3: Former President George H.W. Bush's funeral in Texas: Live stream
Main title: Man Drapes American Flag Outside Bush Family Home
0/3: President Trump, first lady Melani

...

0/3: WATCH: Rogers Man Arrested After Crashing Into Officer’s Car During Pursuit
1/3: 5 Mississippi police officers investigated in custody death
2/3: Fmr. Principal, 90, Arrested For Rape Of Students In The 1960s « CBS Baltimore
Main title: Southlake Swim Coach Found Guilty Of Endangering A Child In Drowning Death « CBS Dallas
0/3: Firefighters Rescue Deer From Backyard Swimming Pool « CBS Pittsburgh
1/3: 5 Mississippi police officers investigated in custody death
2/3: Fmr. Principal, 90, Arrested For Rape Of Students In The 1960s « CBS Baltimore
Main title: ‘Voice of God’ saves men traveling through Ohio from semi-truck crash
0/3: Domino's delivery driver installs fully lit Christmas tree in back of truck
1/3: 2 killed, 3 injured when school bus collides with semi-truck in Illinois
2/3: Leaping lemur surprises Florida trooper during DUI arrest
Main title: Family sues district after daughter was falsely accused of bringing drug-laced candy to school
0/3: Woman sues after spending 3 mo

Main title: Huge queue as new Primark opens in Belfast after fire
0/3: Primark reopens in Belfast after fire
1/3: Egyptians open coffin revealing 3,000-year-old mummified woman
2/3: Homeless Man Turns in $17,000 Discovered Outside Food Bank
Main title: Anything could happen if MPs don't back May's Brexit deal, says Rudd
0/3: Brexit: Amber Rudd says Plan B options plausible
1/3: Amber Rudd floats 'Plan B' before key vote on May's Brexit deal
2/3: May's Brexit vote could plunge the UK into fresh political chaos
Main title: Grace Millane: New Zealand police to charge man with murder
0/3: Man seen with British backpacker Grace Millane to be charged with her murder
1/3: Grace Millane: 'grave fears' held for UK backpacker missing in New Zealand
2/3: Grace Millane: New Zealand Police to charge man with murder
Main title: Airlines ignoring efficient planes in blow to carbon targets – study
0/3: EasyJet plans electric planes by 2030
1/3: Maersk wants to slash carbon emissions from shipping to z

Main title: Mounted Police, APCs Deployed as Act IV of Yellow Vest Protest Unfolds
0/3: In Pictures: Yellow Vest 'Act IV' Protest Violence Spreads Across France
1/3: Livestream: Tear Gas Fired Over Paris Riots
2/3: Mass Arrests In Paris As ‘Yellow Vest’ Protests Enter Violent Third Week, Spread To Belgium
Main title: Migrant Family Border Apprehensions Skyrocket 270 Percent in Nov. over Prior Year
0/3: Migrant Apprehensions at Border Jump to Highest Level Since Trump’s Election
1/3: 18th Street Gang Member Crossed into Texas with Migrant Families, Says Border Patrol
2/3: 120 Migrants Caught at Central Arizona Border in One Day
Main title: Trump Nominates Army Gen. Mark Milley as Chairman of Joint Chiefs
0/3: Trump taps Army head Gen. Mark Milley as Joint Chiefs of Staff chairman
1/3: Donald Trump makes it official: Gen. Mark Milley to chair Joint Chiefs of Staff
2/3: President Trump is expected to name Army Gen. Mark Milley the next Joint Chiefs chairman
Main title: In Pictures: Yellow

Main title: Tanzania Vows Mass Arrests of Alleged Homosexuals
0/3: Tanzania's LGBT community 'fearing for their lives'
1/3: As Tanzania's LGBT fear for their lives, HIV will thrive
2/3: Chinese police shut down 1,100 social media accounts: Xinhua
Main title: Nigeria Denies Anti-Boko Haram Soldiers Begging for Food
0/3: Nigeria Freeing Hundreds of 'Repented' Boko Haram Terrorists
1/3: Nigerian President Denies Dying, Being Replaced with Clone of Himself
2/3: Report: Boko Haram Jihadists Kidnap 18 Girls in Niger
Main title: Al-Shabaab Defector Known as 'Founding Father' Runs for Office
0/3: Former al-Shabab spokesman, Mukhtar Robow, is running for office in Somalia
1/3: War in Africa: U.S. Drops Record Number of Bombs in Somalia in 2018
2/3: U.S. Airstrikes Kill 50 Al-Shabaab Jihadis in Somalia
Main title: Nigerian Army Shoots into Crowd of Shiite Protesters, Killing 27
0/3: Nigeria Denies Anti-Boko Haram Soldiers Begging for Food
1/3: Yellow-vested protesters clash with police forces in

2/3: Donald Trump Welcomes Holocaust Survivors to White House Hanukkah Party
Main title: Dana Perino: My 10 favorite photos of Jasper, America’s Dog – 2018
0/3: College GameDay: The best signs from the Army-Navy game
1/3: A Mauve Colored Brine
2/3: The Special Relationship
Main title: Inside Latin American 'Amazing Race' star’s dramatic escape from Venezuela
0/3: Help Syrians survive the conflict
1/3: Colombia Protests Dozens of Hostages Held in Venezuela
2/3: Mattis: Venezuela’s Maduro a ‘Despot’ Who Will Ultimately ‘Have to Go’
Main title: McDonald's will offer Cheesy Bacon Fries nationwide next year: report
0/3: Ohio State gets bacon vending machine on campus
1/3: Trump seen naming State Department spokeswoman as next UN ambassador
2/3: Police searching for man who went berserk while ordering bagel from deli
Main title: Georgia restaurant employee shoots customer who tried to skip out on bill
0/3: CVS customer uses lengthy store receipt as window-blind slat, as seen in viral photo
1

In [7]:
print(avgArticleLen)

782.4359861591696


In [9]:
sentmatch = []
for i in range(n):
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        comparearticle = wordRepArticles[topmatches[i][j][1]]
        bestmatchSentence =[]
        for k in range(len(mainarticle[2])):
            val = -1
            msentence = mainarticle[2][k][1]
            mainlength = len(mainarticle[2][k][0].split(' '))
            for l in range(len(comparearticle[2])):
                csentence = comparearticle[2][l][1]
                comparelength = len(comparearticle[2][l][0].split(' '))
                commonwords = list(set(msentence.keys()) & set(csentence.keys())) 
                value = 0
                for word in commonwords:
                    mainvalue = (1+math.log(msentence[word]))*1.0/(math.log(1+mainlength/avgArticleLen))
                    compvalue = (1+math.log(csentence[word]))*1.0/(math.log(1+comparelength/avgArticleLen))
                    value += mainvalue*compvalue* math.log(1+n/idf[word])
                if value>val:
                    val = value
                    ind = l
            bestmatchSentence.append([ind,val,comparearticle[2][ind][0]])
    sentmatch.append(bestmatchSentence)

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer() 
sentences = ["VADER is smart, handsome, and funny."]

for i in range(n):
    mainarticle = wordRepArticles[i]
    for j in range(matches):
        val=0
        for k in range(len(mainarticle[2])):
            sent1 = analyzer.polarity_scores(mainarticle[2][k][0])['compound']
            sent2 = analyzer.polarity_scores(sentmatch[i][k][2])['compound']
            val +=sent1*sent2
        

{'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
